<a href="https://colab.research.google.com/github/zak-souid/DLAV_project/blob/main/SiamfcZak_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the required packages.


Clone the repo.

In [4]:
!pip install torch torchvision got10k opencv-python

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


drive/  sample_data/  siamfc-pytorch/


In [7]:
cd drive/MyDrive/'Colab Notebooks'/Project

/content/drive/MyDrive/Colab Notebooks/Project


In [8]:
import os
# get siamfc-pytorch source
if not os.path.exists('siamfc-pytorch'):
  !git clone https://github.com/huanglianghua/siamfc-pytorch.git

Download the pretrained weights.

In [9]:
pretrained_file_name = 'siamfc_alexnet_e50.pth'
if not os.path.exists(pretrained_file_name):
  !gdown https://drive.google.com/uc?id=1UdxuBQ1qtisoWYFZxLgMFJ9mJtGVw6n4

In [10]:
import sys
sys.path.append('siamfc-pytorch/')

Import the necessary libraries.

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import cv2
import os
from collections import namedtuple
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from got10k.trackers import Tracker
from siamfc import ops
from siamfc.backbones import AlexNetV1
from siamfc.heads import SiamFC
from siamfc.losses import BalancedLoss
from siamfc.datasets import Pair
from siamfc.transforms import SiamFCTransforms
from siamfc.siamfc import Net
from siamfc.siamfc import TrackerSiamFC
import glob
from google.colab.patches import cv2_imshow


*Define* the paths for the images, the starting rectangle and the pretrained weights. Then execute the tracker on the images. 

In [17]:
from IPython.display import display, Javascript, Image
from base64 import b64decode, b64encode
import numpy as np
import PIL
import io
import html
import pandas as pd
from google.colab.output import eval_js


In [18]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
        js_reply: JavaScript object containing image from webcam
  Returns:
        img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
        bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [19]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "user"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [24]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
init_img = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    
    if init_img == 0:
      img_0 = js_to_image(js_reply["img"])
      last_img = js_to_image(js_reply["img"])
      init_img =1
    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    pair_img = [img_0, last_img, img]
    net_path = 'siamfc_alexnet_e50.pth'
    tracker = TrackerSiamFC(net_path=net_path)
    boxes, times = tracker.track(pair_img, box_detect, visualize=False)
    #print(boxes[1])
    start_point = (int(boxes[1][0]),int(boxes[1][1]))
    #print(start_point)
    end_point = (int(boxes[1][0])+int(boxes[1][2]),int(boxes[1][1])+int(boxes[1][3]))
    #print(end_point)

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    bbox_array = cv2.rectangle(bbox_array, start_point, end_point, (255, 0, 0), 2) 
    last_img=img


    # draw bounding box of the hand (if multiple hands, just the first one)
    #bbox_array = cv2.rectangle(bbox_array, (168,11), (190,53) , (255, 0, 0), 2) 
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored